In [1]:
import os

path = 'C:/Users/'+os.getlogin()+'/Google Drive/University/Dissertation'
datapath = 'E:/Dissertation Data'
#datapath = 'C:/Users/'+os.getlogin()+'/Dissertation Data'

os.chdir(path+'/Code')
os.getcwd()

'C:\\Users\\User\\Google Drive\\University\\Dissertation\\Code'

In [2]:
import pickle
import pandas as pd
import re
import numpy as np

import nltk
from nltk.tokenize import MWETokenizer

#from glove import Corpus, Glove

from gensim.models import Word2Vec

In [3]:
from nltk.metrics import (
    BigramAssocMeasures,
    TrigramAssocMeasures,
    NgramAssocMeasures,
)

from nltk.metrics.spearman import (
    spearman_correlation,
    ranks_from_scores,
)

We then automatically score the million most strongly associated
n-grams (i.e., roughly the top 5% of the Poisson-ranked
list) for compositionality.

Using word2vec (Mikolov et al., 2013) with the parameters
found to be most effective by Baroni et al. (2014), we
build a word embedding vector for every simplex word in
the vocabulary (ca. 1M types), as well as for each MWE candidate.

* Continuous bag of words model with 400-dimensional vectors, window size 5, subsampling with t = 10^-5, negative sampling with 10 samples. We build vectors only for tokens observed 20 times or more in the corpus.

We then compute the cosine similarity of the vector
representation for a MWE candidate with the vectors of its
constituent words, and take the arithmetic mean. 
In scoring
the compositionality of a candidate, we do not measure the
cosine similarity of the MWE with any stop words it may
contain, as stop words may be assumed to be semantically
uninformative.
* Stop words are taken here to be the 50 most frequent words in the vocabulary.

In [4]:
# Stopwords from corpus - 50 most frequent

with open(datapath+'/Corpora/wiki/simple_20200601/Tagged/stop_clean.pkl', 'rb') as pfile:
    stop = pickle.load(pfile)

In [5]:
stop

{'0|CD',
 '1|CD',
 '2|CD',
 '3|CD',
 '4|CD',
 ':|:',
 'A|DT',
 'He|PRP',
 'In|IN',
 'It|PRP',
 'The|DT',
 'This|DT',
 'also|RB',
 'and|CC',
 'an|DT',
 'are|VBP',
 'as|IN',
 'at|IN',
 'a|DT',
 'be|VB',
 'born|VBN',
 'by|IN',
 'can|MD',
 'first|JJ',
 'for|IN',
 'from|IN',
 'had|VBD',
 'has|VBZ',
 'he|PRP',
 'his|PRP$',
 'in|IN',
 'is|VBZ',
 'it|PRP',
 'not|RB',
 'of|IN',
 'one|CD',
 'on|IN',
 'or|CC',
 'people|NNS',
 'that|IN',
 'that|WDT',
 'their|PRP$',
 'they|PRP',
 'the|DT',
 'to|IN',
 'to|TO',
 'was|VBD',
 'were|VBD',
 'which|WDT',
 'with|IN'}

In [6]:
ngram_eval = pd.read_pickle(datapath+'/Corpora/wiki/simple_20200601/Tagged/ngram_eval_nopn.pkl')

ngram_eval

,ngram,freq,poisson,len,batch
0,"(waystations|NNS, shuku|NN-eki|NN)",20,-607.368914,2,1
1,"(conventionally|RB, delimit|VBP)",20,-638.137312,2,1
2,"(mutually|RB, intelligible|JJ)",20,-641.588322,2,1
3,"(ottava|NN, rima|NN)",21,-649.952795,2,1
4,"(spiny|JJ, dogfish|NN)",20,-655.280617,2,1
...,...,...,...,...,...
149995,"(He|PRP, wrote|VBD, about|IN)",67,-4314.528245,3,3
149996,"(for|IN, every|DT, 1000|CD)",72,-4314.701644,3,8
149997,"(a|DT, combined|VBN)",89,-4314.758077,2,1
149998,"(on|IN, July|NNP, 28|CD)",66,-4315.075053,3,1


In [7]:
min_freq = 20
eval_count = 150000

batch_count = max(ngram_eval.batch)

In [8]:
ngram_eval.batch.value_counts()

 1     45750
 2     35000
 3     21918
 4     16889
 5      9401
 6      7370
 7      4178
 8      4003
 9      2009
-2      1001
 10      827
 11      575
 12      424
-1       369
 13      179
 14       77
 15       30
Name: batch, dtype: int64

In [9]:
from nltk.tokenize import MWETokenizer
from scipy.spatial.distance import cosine
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Flatten down to a single number
def cosim(x,y):
    return cosine_similarity(x.reshape(1,-1), y.reshape(1,-1))[0][0]

In [10]:
def mwe_score(exp, model, stats_frame):
    
    # Combined token for MWE
    mwetoken = '+'.join(exp)

    # Stopwords - 1 if component is a stopword, 0 if present, -1 if simplex word missing from vocab, -2 if MWE missing
    sws = []
    # Component vectors
    cvs = []

    #  Neighbours in original & MWE-aware space
    #oldn = []
    #newn = []

    # List of individual word similarities (where present in the vocab)
    css = []

    # Empty array
    earr = np.empty(400)
    earr[:] = np.nan

    # Check that combined token exists in the vocab. This protects against inflation of n-gram counts caused by repeats
    #  of the same token (e.g. in lists like https://simple.wikipedia.org/wiki/List_of_cities,_towns_and_villages_in_Fars_Province)
    if mwetoken in batch_model.wv.vocab:

        mwv = model.wv[mwetoken]

        for w in exp:
            if w in model.wv.vocab:
                cvs.append(model.wv[w])

                #oldn.append(batch_model.wv.most_similar(w, topn=5))

                if w in stop:
                    sws.append(1)
                    css.append(np.nan)
                else:
                    sws.append(0)
                    css.append(cosim(model.wv[w], mwv ))

            # If component is absent from vocab
            else:
                sws.append(-1)
                cvs.append(earr)
                css.append(np.nan)

                #oldn.append([])

        #  Mean cosim
        if min(sws) >= 0:
            cs = np.nanmean(css)
        else:
            cs = np.nan

        #newn = batch_model.wv.most_similar(mwetoken, topn=5)

    # Combined token missing from vocab - mark with defaults
    else:
        sws = [-2]
        mwv = np.empty(400)
        mwv[:] = np.nan
        cs = np.nan


    # Append to stats df
    return stats_frame.append({
        'ngram'  : exp,
        'stopwords' : sws,
        'mwe_vector' : mwv,
        'component_vectors' : cvs,
        'component_cosims'  : css,
        'cosine_sim'  : cs,
        #'base_nearest': oldn,
        #'mwe_nearest' : newn,
    }, ignore_index=True)

In [11]:
%%time

batch_dfs = {}

for bb in range(batch_count):
    print('Processing batch {} of {}'.format(bb+1,batch_count))
    
    # Subset DataFrame
    batch_dfs[bb] = ngram_eval[ngram_eval.batch == bb+1].reset_index(drop=True)
    
    # Initialise MWETokenizer
    batch_token_mwe = MWETokenizer(list(batch_dfs[bb].ngram) , separator='+')
    
    # Load model
    print('Loading w2v model')

    batch_model = Word2Vec.load(datapath+'/Models/1 w2v/Tagged/simp_tagged_nopn_batch{}.model'.format(bb+1))
    
    print('Gathering MWE stats')
    
    # For each MWE, evaluate stats. Record vectors (in case we want to calculate different metrics later).
    statsf = pd.DataFrame(columns=['ngram', 'stopwords', 'mwe_vector', 'component_vectors', 'component_cosims', 
                                   'cosine_sim']) # , 'base_nearest', 'mwe_nearest'
    batch_len = len(batch_dfs[bb].ngram)
    if batch_len >= 20000: 
        printer = 5000
    elif batch_len >= 5000: 
        printer = 1000
    else:
        printer = 200
        
    _i = 0
    
    for exp in batch_dfs[bb].ngram:
        if _i % printer == 0:
            print(' MWE '+str(_i)+'/'+str(batch_len)+': '+'+'.join(exp))
        _i += 1
        
        statsf = mwe_score(exp, batch_model, statsf)
        
    #  Join back onto DataFrame
    batch_dfs[bb] = batch_dfs[bb].merge(statsf, on='ngram')
    

Processing batch 1 of 15
Loading w2v model
Gathering MWE stats
 MWE 0/45750: waystations|NNS+shuku|NN-eki|NN
 MWE 5000/45750: a|DT+limestone|NN
 MWE 10000/45750: of|IN+Niort|NNP
 MWE 15000/45750: her|PRP$+movie|NN+debut|NN
 MWE 20000/45750: the|DT+wires|NNS
 MWE 25000/45750: opera|NN+called|VBN
 MWE 30000/45750: and|CC+Spike|NNP
 MWE 35000/45750: Aug|NNP+text|NN
 MWE 40000/45750: the|DT+4th|JJ+month|NN
 MWE 45000/45750: of|IN+The|DT+Church|NN
Processing batch 2 of 15
Loading w2v model
Gathering MWE stats
 MWE 0/35000: CA2010|NN+2.4|CD
 MWE 5000/35000: sold|VBN+the|DT
 MWE 10000/35000: pyramid|NN+Government|NNP+Presidential|NNP
 MWE 15000/35000: music|NN+videos|NNS+for|IN
 MWE 20000/35000: that|IN+someone|NN+is|VBZ
 MWE 25000/35000: Organization|NNP+and|CC
 MWE 30000/35000: If|IN+this|DT+is|VBZ
Processing batch 3 of 15
Loading w2v model
Gathering MWE stats
 MWE 0/21918: SIN|NN+JPN|NN
 MWE 5000/21918: across|IN+the|DT+Pacific|NNP
 MWE 10000/21918: the|DT+mid|JJ+to|IN
 MWE 15000/21918: fi

In [12]:
# Merge dataframes, sort by compositionality metric, export

# Also want the default batches with batch no < 0
all_batches = ngram_eval[ngram_eval.batch < 0]

for d in range(batch_count):
    print('Appending batch {} of '.format(d+1)+str(batch_count))
    all_batches = all_batches.append(batch_dfs[d])
    
all_batches = all_batches.sort_values('cosine_sim')
all_batches = all_batches.reset_index(drop=True)

all_batches.to_csv(datapath+'/Models/1 w2v/Tagged/Results/simp_tagged_nopn_output_001.csv', index=False)

Appending batch 1 of 15
Appending batch 2 of 15
Appending batch 3 of 15
Appending batch 4 of 15
Appending batch 5 of 15
Appending batch 6 of 15
Appending batch 7 of 15
Appending batch 8 of 15
Appending batch 9 of 15
Appending batch 10 of 15
Appending batch 11 of 15
Appending batch 12 of 15
Appending batch 13 of 15
Appending batch 14 of 15
Appending batch 15 of 15


In [13]:
all_batches

,ngram,freq,poisson,len,batch,stopwords,mwe_vector,component_vectors,component_cosims,cosine_sim
0,"(was|VBD, born|VBN, here|RB)",32,-2115.954533,3,7,"[1, 1, 0]","[-0.05426647, -0.053498976, -0.15268692, 0.191...","[[0.09795156, -0.35015854, 0.06824661, 0.28937...","[nan, nan, -0.17546517]",-0.175465
1,"(and|CC, finally|RB, the|DT)",33,-2348.089994,3,7,"[1, 0, 1]","[-0.019039303, 0.058172546, -0.07021997, 0.216...","[[-0.1377523, 0.64783734, -0.11971329, 0.08412...","[nan, -0.17055225, nan]",-0.170552
2,"(born|VBN, here|RB)",41,-1946.682114,2,6,"[1, 0]","[-0.005221556, -0.099985056, 0.010824889, 0.15...","[[-0.7368642, -0.7033811, -0.32192045, 1.60515...","[nan, -0.16723849]",-0.167238
3,"(is|VBZ, both|CC, the|DT)",35,-2521.230138,3,7,"[1, 0, 1]","[-0.048536506, -0.11326919, -0.10371729, 0.132...","[[-0.2839466, -0.006846719, -0.32269776, 0.621...","[nan, -0.15586732, nan]",-0.155867
4,"(to|IN, found|VBN, the|DT)",25,-1819.659553,3,2,"[1, 0, 1]","[-0.13175659, -0.014687134, -0.03484833, 0.096...","[[-0.82376164, 0.7050487, -1.2444676, -1.14443...","[nan, -0.14139682, nan]",-0.141397
...,...,...,...,...,...,...,...,...,...,...
149995,"(1|CD, Chemung|NNP, County|NNP)",52,-2990.167422,3,10,"[1, -1, 0]","[-0.10972014, -0.031403285, 0.07398089, -0.068...","[[0.6361498, -0.11636793, -0.17782135, -1.0331...","[nan, nan, 0.6229615]",NaN
149996,"(1|CD, Tioga|NNP, County|NNP)",60,-3442.616347,3,10,"[1, -1, 0]","[-0.2562952, -0.046724327, -0.09596316, -0.065...","[[0.6361498, -0.11636793, -0.17782135, -1.0331...","[nan, nan, 0.63835496]",NaN
149997,"(state|NN, state|NN, state|NN)",48,-2965.054834,3,13,[-2],"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...",[],[],NaN
149998,"(I|PRP, Kissed|VBD, a|DT)",35,-2085.513275,3,14,"[0, -1, 1]","[-0.015156338, -0.079458706, -0.023487035, 0.1...","[[0.2544436, -1.1872422, -0.5404731, -0.401274...","[0.1323022, nan, nan]",NaN


In [14]:
#type(all_batches.stopwords[0])

In [15]:
# RuntimeWarning: Mean of empty slice
#  Appear to be taking the np.nanmean() of a list of NaNs

def inval(inlist,val=0):
    if type(inlist) != list:
        return False
    return val in inlist

#nozero = all_batches[~all_batches.stopwords.apply(inval)]

#minone = nozero[nozero.stopwords.apply(inval,val=-1)]

#minone[minone.stopwords.apply(inval,val=1)]

In [16]:
simp_res_light = all_batches.drop(columns=['mwe_vector', 'component_vectors'])

In [17]:
simp_res_light.to_csv(datapath+'/Models/1 w2v/Tagged/Results/simp_tagged_nopn_light_001.csv', index=False)